In [1]:
import random
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import time

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
### 주소 근사변환 할 함수 정의

In [3]:
def calc_n4(x) :
    n1 = int(x)//10*10
    n2 = int(n1//100*100)
    n3 = int(n2//1000*1000)
    n4 = int(n3/10)
    n5 = int(n4/10)
    n6 = int(n5/10)
    n7 = ''
    n4_list = [n1, n2, n3, n4, n5, n6, n7]
    return  n4_list

def calc_n3(x) :
    n1 = int(x)//10*10
    n2 = int(n1//100*100)
    n3 = int(n2/10)
    n4 = int(n3/10)
    n5 = int(1)
    n6 = ''
    n3_list = [n1, n2, n3, n4, n5, n6]
    return  n3_list

def calc_n2(x) :
    n1 = int(x)//10*10
    n2 = int(n1/10)
    n3 = int(1)
    n4 = ''
    n2_list = [n1, n2, n3, n4]
    return n2_list

def calc_n1() :
    n1 = int(1)
    n2 = ''
    n1_list = [n1, n2]
    return n1_list

In [4]:
start = time.time()
df = pd.read_csv('/Users/parksoyeon/Desktop/22데이터청년캠퍼스/프로젝트/df2.csv')
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace= True)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

browser = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=chrome_options)
browser.get('https://map.naver.com/v5/directions/-/-/-/car')
browser.implicitly_wait(2)
wait = WebDriverWait(browser, random.uniform(1, 1.2))

no_value = browser.find_element(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[2]/button[1]')
elem = browser.find_element(By.XPATH, '//*[@id="directionStart0"]')
elem2 = browser.find_element(By.XPATH, '//*[@id="directionGoal1"]')
elem3 = browser.find_element(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div/div[1]/directions-search/div[2]/button[3]')



for index, rows in tqdm(df.iterrows(), total = (len(df.start_add))):

    # wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionStart0"]')))
    s1 = rows['start_add']
    e1 = rows['end_add']
    # 출발지, 도착지 입력, 경로가 있을 경우
    try:  # 그냥 검색 시도
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionStart0"]')))
        elem.clear()
        elem.send_keys(s1)
        elem.send_keys(Keys.ENTER)

        start_button = browser.find_element(By.CSS_SELECTOR, "button[class='icon_directions start'")
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='icon_directions start'")))
        start_button.click()

    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        # 결과 안나올 경우
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionStart0"]')))
        elem.clear()

        # '-' 구분자가 문자열에 포함된 경우 구분자 없애는 방법 ex. 서초구 방배동 123-24 에서 123만남기고 '-24' 제거
        if '-' in s1:
            s1 = s1.split('-')[0]
        else:
            pass

        elem.send_keys(s1)
        elem.send_keys(Keys.ENTER)

        try:
            start_button = browser.find_element(By.CSS_SELECTOR, "button[class='icon_directions start'")
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='icon_directions start'")))
            start_button.click()
        # 지번이 4째 자리수 일 때

        # -를 떼고난 이후 자리수에 따라 다르게 연산
        except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionStart0"]')))
            juso, number = ' '.join(s1.split(' ')[:-1]), s1.split(' ')[-1]

            if number.isdigit() == True:
                # 4 자리수 연산
                if len(number) == 4:
                    n4_list = calc_n4(number)
                    count = len(n4_list)
                    for num in n4_list:
                        if count >= 0:
                            elem.clear()
                            s1 = juso + ' ' + str(num)
                            elem.send_keys(s1)
                            elem.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                start_button = browser.find_element(By.CSS_SELECTOR,
                                                                    "button[class='icon_directions start'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions start'")))
                                start_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass

                        else:
                            elem.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'
                            break


                elif len(number) == 3:
                    n3_list = calc_n3(number)
                    count = len(n3_list)
                    for num in n3_list:
                        if count >= 0:
                            elem.clear()
                            s1 = juso + ' ' + str(num)
                            elem.send_keys(s1)
                            elem.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                start_button = browser.find_element(By.CSS_SELECTOR,
                                                                    "button[class='icon_directions start'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions start'")))
                                start_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass
                        else:
                            elem.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'
                            break


                elif len(number) == 2:
                    n2_list = calc_n2(number)
                    count = len(n2_list)
                    for num in n2_list:
                        if count >= 0:
                            elem.clear()
                            s1 = juso + ' ' + str(num)
                            elem.send_keys(s1)
                            elem.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                start_button = browser.find_element(By.CSS_SELECTOR,
                                                                    "button[class='icon_directions start'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions start'")))
                                start_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass

                        else:
                            elem.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'

                elif len(number) == 1:
                    n1_list = calc_n1()
                    count = len(n1_list)
                    for num in n1_list:
                        if count >= 0:
                            elem.clear()
                            s1 = juso + ' ' + str(num)
                            elem.send_keys(s1)
                            elem.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                start_button = browser.find_element(By.CSS_SELECTOR,
                                                                    "button[class='icon_directions start'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions start'")))
                                start_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass

                        else:
                            elem.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'
                            break

            else:
                elem.clear()
                s1 = juso
                elem.send_keys(s1)
                elem.send_keys(Keys.ENTER)

                try:
                    start_button = browser.find_element(By.CSS_SELECTOR, "button[class='icon_directions start'")
                    wait.until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='icon_directions start'")))
                    start_button.click()

                except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                    elem.clear()
                    df.at[index, 'road_list'] = 'NaN'
                    df.at[index, 'duration'] = 'NaN'
                    df.at[index, 'fee'] = 'NaN'
                    df.at[index, 'root_dist'] = 'NaN'
                    continue
    # 도착지 작업
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionGoal1"]')))
        elem2.clear()
        elem2.send_keys(e1)
        elem2.send_keys(Keys.ENTER)

        end_button = browser.find_element(By.CSS_SELECTOR, "button[class='icon_directions goal'")
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='icon_directions goal'")))
        end_button.click()

    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        # - 구분자 없애고 시도
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionGoal1"]')))
        elem2.clear()

        # '-' 구분자가 문자열에 포함된 경우 구분자 없애는 방법
        if '-' in e1:
            e1 = e1.split('-')[0]

        else:
            pass

        elem2.send_keys(e1)
        elem2.send_keys(Keys.ENTER)

        try:
            # browser.implicitly_wait(random.uniform(0.1, 0.5))
            # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='icon_directions goal'")))
            end_button = browser.find_element(By.CSS_SELECTOR, "button[class='icon_directions goal'")
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='icon_directions goal'")))
            end_button.click()
        # 지번이 4째 자리수 일 때

        except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="directionGoal1"]')))
            juso, number = ' '.join(e1.split(' ')[:-1]), e1.split(' ')[-1]

            if number.isdigit():
                # 4 자리수 연산
                if len(number) == 4:
                    n4_list = calc_n4(number)
                    count = len(n4_list)
                    for num in n4_list:
                        if count >= 0:
                            elem2.clear()
                            e1 = juso + ' ' + str(num)
                            elem2.send_keys(e1)
                            elem2.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                end_button = browser.find_element(By.CSS_SELECTOR,
                                                                  "button[class='icon_directions goal'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions goal'")))
                                end_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass

                        else:
                            elem2.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'
                            break


                elif len(number) == 3:
                    n3_list = calc_n3(number)
                    count = len(n3_list)
                    for num in n3_list:
                        if count >= 0:
                            elem2.clear()
                            e1 = juso + ' ' + str(num)
                            elem2.send_keys(e1)
                            elem2.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                end_button = browser.find_element(By.CSS_SELECTOR,
                                                                  "button[class='icon_directions goal'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions goal'")))
                                end_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass
                        else:
                            elem2.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'
                            break


                elif len(number) == 2:
                    n2_list = calc_n2(number)
                    count = len(n2_list)
                    for num in n2_list:
                        if count >= 0:
                            elem2.clear()
                            e1 = juso + ' ' + str(num)
                            elem2.send_keys(e1)
                            elem2.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                end_button = browser.find_element(By.CSS_SELECTOR,
                                                                  "button[class='icon_directions goal'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions goal'")))
                                end_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass

                        else:
                            elem2.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'

                elif len(number) == 1:
                    n1_list = calc_n1()
                    count = len(n1_list)
                    for num in n1_list:
                        if count >= 0:
                            elem2.clear()
                            e1 = juso + ' ' + str(num)
                            elem2.send_keys(e1)
                            elem2.send_keys(Keys.ENTER)
                            try:
                                count -= 1
                                end_button = browser.find_element(By.CSS_SELECTOR,
                                                                  "button[class='icon_directions goal'")
                                wait.until(EC.element_to_be_clickable(
                                    (By.CSS_SELECTOR, "button[class='icon_directions goal'")))
                                end_button.click()
                                break
                            except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                                pass

                        else:
                            elem2.clear()
                            df.at[index, 'road_list'] = 'NaN'
                            df.at[index, 'duration'] = 'NaN'
                            df.at[index, 'fee'] = 'NaN'
                            df.at[index, 'root_dist'] = 'NaN'
                            break

            else:
                elem2.clear()
                e1 = juso
                elem.send_keys(e1)
                elem.send_keys(Keys.ENTER)

                try:
                    end_button = browser.find_element(By.CSS_SELECTOR, "button[class='icon_directions goal'")
                    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='icon_directions goal'")))
                    end_button.click()

                except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
                    elem2.clear()
                    df.at[index, 'road_list'] = 'NaN'
                    df.at[index, 'duration'] = 'NaN'
                    df.at[index, 'fee'] = 'NaN'
                    df.at[index, 'root_dist'] = 'NaN'
                    continue


    # 길찾기 클릭
    try:
        browser.implicitly_wait(random.uniform(1, 2))
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "directions-summary-item-car, button[class='icon_directions goal']")))
        elem3.click()

    # 길찾기가 안될 경우 결측처리
    except (NoSuchElementException,TimeoutException) :
        df.at[index, 'road_list'] = 'NaN'
        df.at[index, 'duration'] = 'NaN'
        df.at[index, 'fee'] = 'NaN'
        df.at[index, 'root_dist'] = 'NaN'
        continue

    try :
        browser.implicitly_wait(random.uniform(1, 2))
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'perfect-scrollbar')))
        #time.sleep(random.uniform(0.3, 0.4))
        road = browser.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/directions-summary-list/directions-hover-scroll/div/ul/li[1]/directions-summary-item-car/directions-summary-item-car-card/div[2]/directions-summary-item-car-route-list/ol')
        duration = browser.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/directions-summary-list/directions-hover-scroll/div/ul/li[1]/directions-summary-item-car/directions-summary-item-car-card/div[1]/div[1]/strong')
        fee = browser.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/directions-summary-list/directions-hover-scroll/div/ul/li[1]/directions-summary-item-car/directions-summary-item-car-card/div[1]/div[2]/ul/li[1]/span')
        root_dist = browser.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/directions-summary-list/directions-hover-scroll/div/ul/li[1]/directions-summary-item-car/directions-summary-item-car-card/div[1]/div[1]/span/readable-distance')



    except (NoSuchElementException,TimeoutException) :
        df.at[index, 'road_list'] = 'NaN'
        df.at[index, 'duration'] = 'NaN'
        df.at[index, 'fee'] = 'NaN'
        df.at[index, 'root_dist'] = 'NaN'
        continue


    for roads in road:
        df.at[index, 'road_list'] = roads.text

    for durations in duration :
        df.at[index, 'duration'] = durations.text

    for fees in fee :
        df.at[index, 'fee'] = fees.text

    for root_dists in root_dist :
        df.at[index, 'root_dist'] = root_dists.text

    no_value.click()

    if index % 100 == 0 :

        print(f'{index}번째 데이터 처리중')
        eta = time.time() - start
        print(f"{index}time :", eta)

    if index % 20 == 0 :
        time.sleep(random.uniform(1.5, 2))

/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_72580/1477957633.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=chrome_options)
/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_72580/1477957633.py:8: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=chrome_options)
  0%|                                                  | 0/4000 [00:00<?, ?it/s]

0번째 데이터 처리중
0time : 18.998310327529907


  2%|▉                                     | 100/4000 [03:36<1:33:10,  1.43s/it]

100번째 데이터 처리중
100time : 234.85469317436218


  3%|█                                     | 110/4000 [04:09<2:26:48,  2.26s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=104.0.5112.101)
Stacktrace:
0   chromedriver                        0x000000010b540ef9 chromedriver + 4480761
1   chromedriver                        0x000000010b4cc5d3 chromedriver + 4003283
2   chromedriver                        0x000000010b15f338 chromedriver + 410424
3   chromedriver                        0x000000010b140788 chromedriver + 284552
4   chromedriver                        0x000000010b1b5f7b chromedriver + 765819
5   chromedriver                        0x000000010b1c4b5f chromedriver + 826207
6   chromedriver                        0x000000010b1b2953 chromedriver + 751955
7   chromedriver                        0x000000010b188cd5 chromedriver + 580821
8   chromedriver                        0x000000010b189d25 chromedriver + 584997
9   chromedriver                        0x000000010b51202d chromedriver + 4288557
10  chromedriver   

In [6]:
df.to_csv('/Users/parksoyeon/Desktop/22데이터청년캠퍼스/프로젝트/airport_filtered_df2_day.csv', index = False, encoding ='utf-8-sig')

In [ ]:
import os
os.system('shutdown -s -f')